# How to write multi-batch Batch Requests
* How do you specify the Mult-Batch BatchRequests
What is left to do? 
- we can try to do the different Data Connenctors
    - Configured
    - Inferre
- it's a little hand tied becas

# let's do a brain dump before signing off 

In [1]:
import great_expectations as ge

from ruamel import yaml

from great_expectations.core.batch import BatchRequest

from great_expectations.rule_based_profiler.rule.rule import Rule
from great_expectations.rule_based_profiler.rule_based_profiler import RuleBasedProfiler, RuleBasedProfilerResult

from great_expectations.rule_based_profiler.domain_builder import (
    DomainBuilder,
    ColumnDomainBuilder,
)
from great_expectations.rule_based_profiler.parameter_builder import (
    MetricMultiBatchParameterBuilder,
)
from great_expectations.rule_based_profiler.expectation_configuration_builder import (
    DefaultExpectationConfigurationBuilder,
)

In [3]:
data_context: ge.DataContext = ge.get_context()

In [4]:
data_path: str = "../../../../test_sets/taxi_yellow_tripdata_samples"

datasource_config = {
    "name": "taxi_multi_batch_datasource",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "module_name": "great_expectations.execution_engine",
        "class_name": "PandasExecutionEngine",
    },
    "data_connectors": {
        "default_inferred_data_connector_name": {
            "class_name": "InferredAssetFilesystemDataConnector",
            "base_directory": data_path,
            "default_regex": {
                "group_names": ["data_asset_name", "month"],
                "pattern": "(yellow_tripdata_sample_2018)-(\\d.*)\\.csv",
            },
        },
        "default_inferred_data_connector_name_all_years": {
            "class_name": "InferredAssetFilesystemDataConnector",
            "base_directory": data_path,
            "default_regex": {
                "group_names": ["data_asset_name", "year", "month"],
                "pattern": "(yellow_tripdata_sample)_(\\d.*)-(\\d.*)\\.csv",
            },
        },
    },
}

data_context.test_yaml_config(yaml.dump(datasource_config))

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_inferred_data_connector_name : InferredAssetFilesystemDataConnector

	Available data_asset_names (1 of 1):
		yellow_tripdata_sample_2018 (3 of 12): ['yellow_tripdata_sample_2018-01.csv', 'yellow_tripdata_sample_2018-02.csv', 'yellow_tripdata_sample_2018-03.csv']

	Unmatched data_references (3 of 30):['.DS_Store', 'first_3_files', 'random_subsamples']

	default_inferred_data_connector_name_all_years : InferredAssetFilesystemDataConnector

	Available data_asset_names (1 of 1):
		yellow_tripdata_sample (3 of 36): ['yellow_tripdata_sample_2018-01.csv', 'yellow_tripdata_sample_2018-02.csv', 'yellow_tripdata_sample_2018-03.csv']

	Unmatched data_references (3 of 6):['.DS_Store', 'first_3_files', 'random_subsamples']



In [10]:
In this example we have two DataConnectors that connect to to our Database. 
- what are the methods that we can use? 
- get_Validator() can we do it for something that is multi-batch?
- what about run profiler? 
- get batch_list?


SyntaxError: invalid syntax (4013422935.py, line 1)

In [5]:
# add_datasource only if it doesn't already exist in our configuration
try:
    data_context.get_datasource(datasource_config["name"])
except ValueError:
    data_context.add_datasource(**datasource_config)

In [8]:
single_batch_batch_request: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_datasource",
    data_connector_name="default_inferred_data_connector_name",
    data_asset_name="yellow_tripdata_sample_2018",
    data_connector_query={"index": -1},
)

In [9]:
multi_batch_batch_request: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_datasource",
    data_connector_name="default_inferred_data_connector_name",
    data_asset_name="yellow_tripdata_sample_2018",
)

In [11]:
data_context.get_batch_list(batch_request=single_batch_batch_request)

In [18]:
yes = data_context.get_batch_list(batch_request=multi_batch_batch_request)

In [13]:
# how do you then inspect a batch? 


In [14]:
# how do you then put it thruogh a profiler? 

In [16]:
data_context.get_batch(batch_request=multi_batch_batch_request) # this will give an error

/Users/work/Development/great_expectations/great_expectations/data_context/data_context/base_data_context.py:924: DeprecationWarning: get_batch is deprecated for the V3 Batch Request API as of v0.13.20 and will be removed in v0.16. Please use get_batch_list instead.
  warnings.warn(


ValueError: Got 12 batches instead of a single batch. If you would like to use a BatchRequest to return multiple batches, please use get_batch_list directly instead of calling get_batch

In [20]:
my_expectation_suite = data_context.create_expectation_suite(expectation_suite_name="temp", overwrite_existing=True)

In [21]:
validator = data_context.get_validator_using_batch_list(batch_list = yes, expectation_suite =my_expectation_suite )

In [22]:
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2,2018-12-22 18:30:39,2018-12-22 18:42:37,1,1.39,1,N,170,229,2,9.0,0.0,0.5,0.00,0.0,0.3,9.80,NaN
1,2,2018-12-29 14:46:47,2018-12-29 15:07:41,1,3.77,1,N,68,140,1,16.0,0.0,0.5,5.04,0.0,0.3,21.84,NaN
2,1,2018-12-01 16:04:05,2018-12-01 16:45:20,1,4.90,1,N,263,249,1,26.5,0.0,0.5,5.46,0.0,0.3,32.76,NaN
3,1,2018-12-31 15:28:07,2018-12-31 15:28:16,0,0.00,5,N,132,132,1,70.0,0.0,0.0,0.00,0.0,0.3,70.30,NaN
4,2,2018-12-31 18:13:34,2018-12-31 18:41:03,1,6.74,1,N,162,116,1,24.5,1.0,0.5,5.26,0.0,0.3,31.56,NaN


In [29]:
validator.batches

OrderedDict([('b6f920d6002d3c556cb1e59f6ab9ad0a',
              <great_expectations.core.batch.Batch at 0x7fd941b7fe20>),
             ('b35c6659eef933e59905d3145c7cba81',
              <great_expectations.core.batch.Batch at 0x7fd941b84310>),
             ('36da19040ac7ba6180d922745a227b30',
              <great_expectations.core.batch.Batch at 0x7fd941808160>),
             ('16477b79ef22972db9576c9e1748add3',
              <great_expectations.core.batch.Batch at 0x7fd941d944c0>),
             ('64c03aae33d67af09a8ce4b7e82b60e2',
              <great_expectations.core.batch.Batch at 0x7fd942184880>),
             ('50bfcc4ddd0935edddbe82c498b7d405',
              <great_expectations.core.batch.Batch at 0x7fd941b7e6d0>),
             ('cc7cc6b9e3e2ca9d029b1190f0fcba74',
              <great_expectations.core.batch.Batch at 0x7fd9417ff5e0>),
             ('142dd8def2b56dcea0d4ab00cb59ceba',
              <great_expectations.core.batch.Batch at 0x7fd941804e80>),
             ('ace0a35cb

In [38]:
my_batch=list(validator.batches.values())[0]

In [41]:
my_batch.to_dict()

{'data': '<great_expectations.execution_engine.pandas_batch_data.PandasBatchData object at 0x7fd941789100>',
 'batch_request': {'datasource_name': 'taxi_multi_batch_datasource',
  'data_connector_name': 'default_inferred_data_connector_name',
  'data_asset_name': 'yellow_tripdata_sample_2018',
  'data_connector_query': None,
  'limit': None,
  'batch_spec_passthrough': None},
 'batch_definition': {'datasource_name': 'taxi_multi_batch_datasource',
  'data_connector_name': 'default_inferred_data_connector_name',
  'data_asset_name': 'yellow_tripdata_sample_2018',
  'batch_identifiers': {'month': '01'}},
 'batch_spec': {'path': '/Users/work/Development/great_expectations/tests/test_fixtures/rule_based_profiler/example_notebooks/great_expectations/../../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2018-01.csv'},
 'batch_markers': {'ge_load_time': '20220713T235243.398401Z',
  'pandas_data_fingerprint': '7a0d51e71ca42b9d4be20b776eba0cf0'}}

In [27]:
validator.expect_column_to_exist("i_dont_exist")

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "result": {},
  "success": false,
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [28]:
# is this across everything? ?
# well no. 

In [ ]:
What can you do with multiple batches? 
* 